In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def reduce_mem_usage(df):
    """
    reduce the memory of dataframe 
    """
    start_mem = df.memory_usage().sum() 
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() 
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
# load user and item data
df_item = pd.read_csv("./data/tianchi_mobile_recommend_train_item.csv")
df_user = pd.read_csv("./data/tianchi_mobile_recommend_train_user.csv")
df_item = reduce_mem_usage(df_item)
df_user = reduce_mem_usage(df_user)
df_item.head()

In [ ]:
item_data = pd.read_csv("./data/tianchi_mobile_recommend_train_item.csv")
user_data = pd.read_csv("./data/tianchi_mobile_recommend_train_user.csv")

In [ ]:
user_data.head()

In [ ]:
user_data.isnull().sum()/len(user_data)

In [ ]:
item_data.head()

In [ ]:
item_data.isnull().sum()/len(item_data)

In [ ]:
item_list = item_data["item_id"].unique().tolist()

In [ ]:
len(user_data[user_data["behavior_type"] == 4])/len(user_data)

In [ ]:
user_data["time"] = pd.to_datetime(user_data["time"], format="%Y%m%d %H")
user_data["date"] = user_data["time"].dt.date
user_data["weekday"] = user_data["time"].dt.weekday
user_data["hour"] = user_data["time"].dt.hour


In [ ]:
# plot everyday buy counts
def erveryday_buy_counts(in_itemlist=False):
    if in_itemlist == True:
        condition = user_data["item_id"].isin(item_list)
    else:
        condition = True
    item_day_buy_count = user_data[(user_data["behavior_type"] == 4)&(condition)][["user_id", "item_id", "date"]].drop_duplicates()
    item_day_buy_count["buy_count"] = 1
    item_day_buy_count["date"] = item_day_buy_count["date"].apply(lambda x : str(x)[5:])
    item_day_buy_count = item_day_buy_count[["date", "buy_count"]].groupby("date", as_index=False).count().sort_values(by="date")
    f, ax = plt.subplots(figsize=(20, 5))
    fig = sns.barplot(x=item_day_buy_count["date"], y=item_day_buy_count["buy_count"])

In [ ]:
erveryday_buy_counts(in_itemlist=False)

In [ ]:
erveryday_buy_counts(in_itemlist=True)

In [ ]:
def weekday_buy_counts(type, in_itemlist=False):
    if in_itemlist == True:
        condition = user_data["item_id"].isin(item_list)
    else:
        condition = True
    weekday_buy_counts = user_data[(user_data["behavior_type"] == type)&(condition)][["weekday"]]
    weekday_buy_counts["count"] = 1
    weekday_buy_counts["weekday"] = weekday_buy_counts["weekday"].apply(lambda x: x+1)
    weekday_buy_counts = weekday_buy_counts.groupby("weekday", as_index=False).count().sort_values(by="count", ascending=False).reset_index(drop=True)
    return weekday_buy_counts
def hour_buy_counts(type, in_itemlist=False):
    if in_itemlist == True:
        condition = user_data["item_id"].isin(item_list)
    else:
        condition = True
    hour_buy_counts = user_data[(user_data["behavior_type"] == type)&(condition)][["hour"]]
    hour_buy_counts["count"] = 1
    hour_buy_counts = hour_buy_counts.groupby("hour", as_index=False).count().sort_values(by="count", ascending=False).reset_index(drop=True)
    return hour_buy_counts

In [ ]:
data = weekday_buy_counts(4, False)
fig = sns.barplot(x="weekday", y="count", data=data)

In [ ]:
data = weekday_buy_counts(4, True)
fig = sns.barplot(x="weekday", y="count", data=data)


In [ ]:
data = hour_buy_counts(4,True)
fig = sns.barplot(x="hour", y="count", data=data)

In [ ]:
data = hour_buy_counts(4,True)
fig = sns.barplot(x="hour", y="count", data=data)